# Урок 3. Part-of-Speech разметка, NER , извлечение отношений.

### Тема “Улучшение качества  POS-taggera”

На вебинаре был рассмотрен пример тегирования с использованием корпуса на Русском языке, вам необходимо улучшить модель что бы качество классификации было выше чем с лог регрессией 

### Тема “Создание признакового пространства”

Используем предобработанные в рамках 1-ого домашнего задания датасет combine_df_prepocessed.pkl. Используем столбец 'clean_tweet'.
##### Задание 1.
Используя библиотеку Spacy, вывести ТОП-20 популярных NER в combine_df датасете. Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? (Учтите, что max_word_limit_spacy для Spacy = 1000000)


С помощью Spacy выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. Действительно ли в топ вошли только персоны и организации или есть мусор?


Повторим шаги из заданий 1 и 2, используя библиотеку nltk.
##### Задание 2.
Используя библиотеку nltk, вывести ТОП-20 популярных NER в combine_df датасете. Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? Для данного задания используем ограничение на количество символов во входном датасете (max_word_limit_spacy = 1000000), чтобы иметь возможность сравнить результаты работы Spacy и nltk. Обратите внимание, что nltk чувствителен к регистру.


С помощью nltk выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. Действительно ли в топ вошли только персоны и организации или есть мусор?

##### Задание 3.
Какая из библиотек по вашему лучше отработала? Сравните качество полученных most_common NER и количество распознаных NER.


In [3]:
import pandas as pd
import numpy as np

import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
import os

In [4]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 800)

### Тема “Улучшение качества  POS-taggera”

На вебинаре был рассмотрен пример тегирования с использованием корпуса на Русском языке, вам необходимо улучшить модель что бы качество классификации было выше чем с лог регрессией

In [5]:
import pyconll

from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.metrics import accuracy_score

In [6]:
full_train = pyconll.load_from_file('datasets/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('datasets/ru_syntagrus-ud-dev.conllu')

In [7]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [8]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [9]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)
test_enc_labels = le.transform(test_label)

In [10]:
hvectorizer = HashingVectorizer(ngram_range=(1, 3), analyzer='char', n_features=100)
X_train = hvectorizer.fit_transform(train_tok)
X_test = hvectorizer.transform(test_tok)

##### Вариант обучения на логистической регрессии

In [11]:
lr = LogisticRegression(random_state=0, max_iter=100)
lr.fit(X_train, train_enc_labels)
pred = lr.predict(X_test)
accuracy_score(test_enc_labels, pred)

C:\Users\rseme\.conda\envs\tf2-gpu\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


0.7213460047854953

##### Вариант обучения на RandomForestClassifier

In [12]:
from sklearn.ensemble import RandomForestClassifier

lgb = RandomForestClassifier(
    n_estimators=10,
    verbose=False,
    n_jobs=8,
    random_state=27
)
lgb.fit(X_train, train_enc_labels)
pred = lgb.predict(X_test)
accuracy_score(test_enc_labels, pred)

0.942818387085903

### Тема “Создание признакового пространства”

Используем предобработанные в рамках 1-ого домашнего задания датасет combine_df_prepocessed.pkl. Используем столбец 'clean_tweet'.
##### Задание 1.
Используя библиотеку Spacy, вывести ТОП-20 популярных NER в combine_df датасете. Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? (Учтите, что max_word_limit_spacy для Spacy = 1000000)

С помощью Spacy выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. Действительно ли в топ вошли только персоны и организации или есть мусор?

In [82]:
import spacy
import en_core_web_md
from spacy import displacy
from collections import Counter

In [14]:
data = pd.read_pickle("lesson_01.pkl")
data.head(2)

,id,label,tweet,prep_tweet,tweet_token,tweet_token_filtered,tweet_stemmed,tweet_lemmatized
0,1,0.0,@user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction. #run,when father is dysfunctional and is so selfish he drags his kids into his dysfunction run,"[when, father, is, dysfunctional, and, is, so, selfish, he, drags, his, kids, into, his, dysfunction, run]","[father, dysfunctional, selfish, drags, kids, dysfunction, run]","[father, dysfunct, selfish, drag, kid, dysfunct, run]","[father, dysfunctional, selfish, drag, kid, dysfunction, run]"
1,2,0.0,@user @user thanks for #lyft credit i can't use cause they don't offer wheelchair vans in pdx. #disapointed #getthanked,thanks for lyft credit cannot use cause they do not offer wheelchair vans in pdx disapointed getthanked,"[thanks, for, lyft, credit, can, not, use, cause, they, do, not, offer, wheelchair, vans, in, pdx, disapointed, getthanked]","[thanks, lyft, credit, use, cause, offer, wheelchair, vans, pdx, disapointed, getthanked]","[thank, lyft, credit, use, caus, offer, wheelchair, van, pdx, disapoint, getthank]","[thank, lyft, credit, use, cause, offer, wheelchair, vans, pdx, disapointed, getthanked]"


In [83]:
nlp = en_core_web_md.load()
ner = data['prep_tweet'].apply(lambda x: [[doc.ent_type_, doc.text ] for doc in nlp(x) if doc.ent_type_])

In [84]:
c_top20 = []
c_ner = []
c_person = []
c_org = []

for row in ner:
    for el in row:
        c_top20.append(f'{el[1]} ({el[0]})')
        c_ner.append(el[0])
        if el[0] == 'PERSON':
            c_person.append(f'{el[1]} ({el[0]})')
        if el[0] == 'ORG':
            c_org.append(f'{el[1]} ({el[0]})')

##### ТОП-20 популярных NER

In [85]:
Counter(c_top20).most_common(20)

[('today (DATE)', 1438),
 ('day (DATE)', 1276),
 ('friday (DATE)', 752),
 ('the (DATE)', 615),
 ('tomorrow (DATE)', 612),
 ('morning (TIME)', 611),
 ('weekend (DATE)', 561),
 ('summer (DATE)', 555),
 ('one (CARDINAL)', 548),
 ('sunday (DATE)', 546),
 ('week (DATE)', 546),
 ('days (DATE)', 539),
 ('first (ORDINAL)', 512),
 ('yoyour (GPE)', 499),
 ('this (DATE)', 498),
 ('orlando (GPE)', 497),
 ('tonight (TIME)', 431),
 ('year (DATE)', 409),
 ('night (TIME)', 403),
 ('bihday (DATE)', 355)]

In [86]:
Counter(c_ner).most_common(1)

[('PERSON', 22462)]

In [88]:
Counter(c_person).most_common(20)

[('bihday (PERSON)', 334),
 ('bing (PERSON)', 214),
 ('bong (PERSON)', 214),
 ('obama (PERSON)', 131),
 ('hillary (PERSON)', 88),
 ('th (PERSON)', 88),
 ('suppo (PERSON)', 83),
 ('sta (PERSON)', 81),
 ('bull (PERSON)', 79),
 ('trump (PERSON)', 71),
 ('donald (PERSON)', 64),
 ('hu (PERSON)', 62),
 ('christina (PERSON)', 57),
 ('am (PERSON)', 56),
 ('usd (PERSON)', 45),
 ('sun (PERSON)', 44),
 ('bjp (PERSON)', 44),
 ('john (PERSON)', 42),
 ('grimmie (PERSON)', 41),
 ('cox (PERSON)', 41)]

In [89]:
Counter(c_org).most_common(20)

[('buffalo (ORG)', 189),
 ('allahsoil (ORG)', 110),
 ('simulation (ORG)', 106),
 ('sjw (ORG)', 102),
 ('the (ORG)', 98),
 ('orlando (ORG)', 88),
 ('trump (ORG)', 78),
 ('gop (ORG)', 78),
 ('bihday (ORG)', 76),
 ('polar (ORG)', 68),
 ('bull (ORG)', 65),
 ('islam (ORG)', 64),
 ('nba (ORG)', 62),
 ('day (ORG)', 58),
 ('sun (ORG)', 54),
 ('hill (ORG)', 54),
 ('social (ORG)', 53),
 ('news (ORG)', 51),
 ('ht (ORG)', 50),
 ('new (ORG)', 49)]

Повторим шаги из заданий 1 и 2, используя библиотеку nltk.

##### Задание 2.
Используя библиотеку nltk, вывести ТОП-20 популярных NER в combine_df датасете. Какой тип NER (ORG, GPE, PERSON и тд) оказался самым популярным? Для данного задания используем ограничение на количество символов во входном датасете (max_word_limit_spacy = 1000000), чтобы иметь возможность сравнить результаты работы Spacy и nltk. Обратите внимание, что nltk чувствителен к регистру.

С помощью nltk выяснить: какие персоны и организации самые обсуждаемые в train и test датасетах? вывести ТОП-20 самых популярных. Действительно ли в топ вошли только персоны и организации или есть мусор?

##### Задание 3.
Какая из библиотек по вашему лучше отработала? Сравните качество полученных most_common NER и количество распознаных NER.